<a href="https://colab.research.google.com/github/once-upon-an-april/Thuc-Hanh-Deep-Learning-trong-Khoa-Hoc-Du-Lieu-DS201.Q11.1/blob/main/22520975_Lab2_Bai4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------------------------------------------------------------------
# 0. THIẾT LẬP VÀ CÀI ĐẶT
# -------------------------------------------------------------------
print("Bài 4: Đang cài đặt/nâng cấp thư viện (transformers, accelerate...)...")
# Cài đặt/Nâng cấp các thư viện chính để đồng bộ phiên bản
!pip install -q --upgrade transformers accelerate datasets evaluate timm scikit-learn torch torchvision

In [ ]:
import torch
import numpy as np
import os
import time
from tqdm.auto import tqdm
from google.colab import drive
import transformers
import datasets
import evaluate
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from huggingface_hub import HfFolder, login

In [ ]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
DATA_DIR = '/content/drive/MyDrive/Data/VinaFood21/VinaFood21'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Bài 4: Sử dụng thiết bị: {device}")

if not os.path.exists(DATA_DIR):
    print(f"LỖI: Không tìm thấy thư mục '{DATA_DIR}'.")

NUM_CLASSES = 21
NUM_EPOCHS_B4 = 10
BATCH_SIZE = 32

In [ ]:
if __name__ == "__main__" and 'transformers' in globals() and os.path.exists(os.path.join(DATA_DIR, 'train')):
    # 1. Tải dataset
    print("Bài 4: Đang tải VinaFood21 bằng HuggingFace datasets...")
    dataset = load_dataset("imagefolder", data_dir=DATA_DIR)

    # 2. Lấy nhãn
    labels = dataset["train"].features["label"].names
    label2id, id2label = {}, {}
    for i, label in enumerate(labels):
        label2id[label] = str(i)
        id2label[str(i)] = label
    print(f"Bài 4: Phát hiện {len(labels)} lớp.")

    # 3. Tải Image Processor
    model_checkpoint = "microsoft/resnet-50"
    processor = AutoImageProcessor.from_pretrained(model_checkpoint)

    # 4. Hàm transform (Lazy)
    def transform_hf_lazy(examples):
        images = [img.convert("RGB") for img in examples["image"]]
        inputs = processor(images, return_tensors="pt")
        # SỬA: Đổi tên key thành 'labels' (số nhiều)
        inputs["labels"] = examples["label"]
        return inputs

    # SỬA: Dùng with_transform (lazy loading) để khởi động nhanh
    dataset = dataset.with_transform(transform_hf_lazy)

    # 5. Data collator (SỬA LỖI TypeError)
    # Collator này sẽ lọc dict, chỉ lấy 2 key cần thiết
    def collate_fn(examples):
        # 'examples' là list các dict từ transform
        pixel_values = torch.stack([example["pixel_values"] for example in examples])
        labels = torch.tensor([example["labels"] for example in examples]) # Lấy 'labels' (số nhiều)

        # Chỉ trả về dict mà Trainer/Model hiểu
        return {"pixel_values": pixel_values, "labels": labels}

    # 6. Tải mô hình
    print("Bài 4: Đang tải mô hình ResNet-50 từ HuggingFace...")
    model_hf = AutoModelForImageClassification.from_pretrained(
        model_checkpoint,
        num_labels=NUM_CLASSES,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )

    # 7. Hàm tính metrics
    def compute_metrics_hf(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro', zero_division=0)
        acc = accuracy_score(labels, predictions)
        return {'accuracy': acc, 'precision_macro': precision, 'recall_macro': recall, 'f1_macro': f1}

    # 8. Training Arguments
    if "train" in dataset:
        training_args = TrainingArguments(
            output_dir="./vinafood_resnet50_hf",
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,

            # Sử dụng tham số MỚI
            eval_strategy="epoch",
            save_strategy="epoch",

            num_train_epochs=NUM_EPOCHS_B4,
            load_best_model_at_end=True,
            logging_steps=100,
            optim="adamw_torch",
            learning_rate=2e-5,

            # TĂNG TỐC ĐỘ: Bật mixed precision (cho L4/T4)
            fp16=True,

            # SỬA LỖI KeyError: Đặt là False
            remove_unused_columns=False,

            report_to="none",
        )

        # 9. Khởi tạo Trainer
        trainer = Trainer(
            model=model_hf,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["test"],
            tokenizer=processor,
            compute_metrics=compute_metrics_hf,
            data_collator=collate_fn, # SỬ DỤNG COLLATOR ĐÃ SỬA
        )

        # 10. Huấn luyện
        print(f"\nBài 4: Huấn luyện ResNet-50 (HuggingFace) trong {NUM_EPOCHS_B4} epochs...")
        trainer.train()

        # 11. Đánh giá
        print("\n[Bài 4: ResNet-50] Đánh giá cuối cùng:")
        eval_results = trainer.evaluate(dataset["test"])

        print("\n--- KẾT QUẢ ĐÁNH GIÁ (HuggingFace) ---")
        print(f"Accuracy:  {eval_results['eval_accuracy']:.4f}")
        print(f"Precision: {eval_results['eval_precision_macro']:.4f}")
        print(f"Recall:    {eval_results['eval_recall_macro']:.4f}")
        print(f"F1-Score:  {eval_results['eval_f1_macro']:.4f}")
        print("-" * 39)

    else:
        print("Bài 4: dataset['train'] không tồn tại.")
elif 'transformers' in globals():
    print("Bỏ qua Bài 4 do không tải được dữ liệu (Kiểm tra DATA_DIR).")

print("\n" + "="*50)
print("BÀI 4 ĐÃ HOÀN TẤT.")
print("="*50)